In [24]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
from rdflib.plugins.sparql import prepareQuery

# Load the ontology
g = Graph()
g.parse("FinalNewRdfXml.rdf")

# Define the namespace
movie_proj = Namespace("http://www.semanticweb.org/ranaahmed/ontologies/2024/4/Movei-Project")

# Define inference rules using SPARQL queries
inference_rules = [
    # Rule 1: Movies with at least one actor and one director
    """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX movie_proj: <http://www.semanticweb.org/ranaahmed/ontologies/2024/4/Movei-Project/>
    INSERT {
        ?movie rdf:type movie_proj:MovieWithActorsAndDirectors .
    }
    WHERE {
        ?movie a movie_proj:Movie .
        ?movie movie_proj:hasActor ?actor .
        ?movie movie_proj:hasDirector ?director .
    }
    """,
    # Rule 2: Movies where the actor and director are the same person
    """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX movie_proj: <http://www.semanticweb.org/ranaahmed/ontologies/2024/4/Movei-Project/>
    INSERT {
        ?movie rdf:type movie_proj:SameActorAndDirector .
    }
    WHERE {
        ?movie a movie_proj:Movie .
        ?movie movie_proj:hasActor ?person .
        ?movie movie_proj:hasDirector ?person .
    }
    """,
    # Rule 3: Recent Movies (Released after 2010)
    """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX movie_proj: <http://www.semanticweb.org/ranaahmed/ontologies/2024/4/Movei-Project/>
    INSERT {
        ?movie rdf:type movie_proj:RecentMovie .
    }
    WHERE {
        ?movie a movie_proj:Movie .
        ?movie movie_proj:year ?year .
        FILTER (?year > 2010)
    }
    """
]

# Apply inference rules using SPARQL queries
for rule in inference_rules:
    g.update(rule, initNs={"movie_proj": movie_proj})

# SPARQL query to retrieve inferred movie names for Rule 1
query_movie_names_rule1 = prepareQuery("""
    PREFIX movie_proj: <http://www.semanticweb.org/ranaahmed/ontologies/2024/4/Movei-Project/>
    SELECT DISTINCT ?movieName
    WHERE {
        ?movie a movie_proj:MovieWithActorsAndDirectors .
        ?movie movie_proj:title ?movieName .
    }
""", initNs={"movie_proj": movie_proj})

# Retrieve inferred movie names for Rule 1
unique_movie_names_rule1 = set()
for row in g.query(query_movie_names_rule1):
    unique_movie_names_rule1.add(row.movieName)

# Print inferred movie names for Rule 1
print("Rule 1: Movies with at least one actor and one director")
for movie_name in unique_movie_names_rule1:
    print(movie_name)

# SPARQL query to retrieve inferred movie names for Rule 2
query_movie_names_rule2 = prepareQuery("""
    PREFIX movie_proj: <http://www.semanticweb.org/ranaahmed/ontologies/2024/4/Movei-Project/>
    SELECT DISTINCT ?movieName
    WHERE {
        ?movie a movie_proj:SameActorAndDirector .
        ?movie movie_proj:title ?movieName .
    }
""", initNs={"movie_proj": movie_proj})

# Retrieve inferred movie names for Rule 2
unique_movie_names_rule2 = set()
for row in g.query(query_movie_names_rule2):
    unique_movie_names_rule2.add(row.movieName)

# Print inferred movie names for Rule 2
print("--------------------------------------------")
print("Rule 2: Movies where the actor and director are the same person")
for movie_name in unique_movie_names_rule2:
    print(movie_name)

# SPARQL query to retrieve inferred movie names for Rule 3
query_recent_movies = prepareQuery("""
    PREFIX movie_proj: <http://www.semanticweb.org/ranaahmed/ontologies/2024/4/Movei-Project/>
    SELECT DISTINCT ?movieName
    WHERE {
        ?movie a movie_proj:RecentMovie .
        ?movie movie_proj:title ?movieName .
    }
""", initNs={"movie_proj": movie_proj})

# Retrieve inferred movie names for Rule 3
unique_recent_movie_names = set()
for row in g.query(query_recent_movies):
    unique_recent_movie_names.add(row.movieName)

# Print inferred movie names for Rule 3
print("--------------------------------------------")
print("Rule 3: Recent Movies (Released after 2010)")
for movie_name in unique_recent_movie_names:
    print(movie_name)


Rule 1: Movies with at least one actor and one director
Pulp Fiction
The Dark Knight
Parasite
FightClub
Snatch
Heat
Reservoir Dogs
Kill Bill (volume 1)
The Silence Of The Lambs
Life Is Beautiful
--------------------------------------------
Rule 2: Movies where the actor and director are the same person
Pulp Fiction
Parasite
FightClub
Reservoir Dogs
Kill Bill (volume 1)
Life Is Beautiful
--------------------------------------------
Rule 3: Recent Movies (Released after 2010)
Parasite
